# Projeto 1 - Ciência dos Dados

Nome: Augusto Rocha Ribeiro

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
#-------------------------------------Importando as bibliotecas necessárias-----------------------------------------------------

import pandas as pd
import os
import re 
from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer()

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
#### ----------------------------------Pegando o excel com as classificações manuais dos tweets-------------------------------------

filename = 'Havaianas.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com o projeto!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo Havaianas.xlsx, tudo certo para prosseguir com o projeto!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
#----------------------------------------Definindo Relevantes e Irrelevantes----------------------------------------------------

train = pd.read_excel(filename)
train_relevante = train[train['Classificação']==1]
train_irrelevante = train[train['Classificação']==0]

test = pd.read_excel(filename, sheet_name = 'Teste')
test_relevante = test[test['Classificação']==1]
test_irrelevante = test[test['Classificação']==0]


#------------------------------------------Classifica as categorias-------------------------------------------------------------

train['Classificação'] = train['Classificação'].astype('category')
test['Classificação'] = test['Classificação'].astype('category')

train.Classificação.cat.categories = ['Irrelevante', 'Relevante']
test.Classificação.cat.categories = ['Irrelevante', 'Relevante']

___
## Classificador automático de sentimento


Foram considerados relevantes os comentários que expressavam uma crítica sobre o produto, seja ela positiva ou negativa.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [4]:
# Função para retirada de caracteres indesejado:

def cleanup(text):
    punctuation = '[!-.:?;@/""“”,_]'
    pattern = re.compile(punctuation)
    text_pronto = re.sub(pattern, '', text)
    return text_pronto


# Função para retirada dos acentos nos textos:

def tira_acentos(texto):
    acentos = {'a':['á', 'à', 'ã', 'â', 'ä'], 'e':['ê', 'é', 'è', 'ë'], 'i':['í', 'ì', 'î', 'ï'], 'o':['ó', 'ò', 'ô', 'õ', 'ö'], 'u':['ú', 'ù', 'û', 'ü'], 'c':['ç']} 
    for l in acentos:
        for e in acentos[l]:
            texto = texto.replace(e, l)
    return texto


#----------------------------Cria uma string com todas as palavras e caracteres da parte de Treinamento-------------------------

tweets_train = ''
for i in range(len(train)):
    tweets_train += train['Treinamento'].loc[i]

train_limpo = tira_acentos(cleanup(tweets_train.lower()))    # Deixa a string somente com letras minúsculas, retira caracteres
                                                             # indesejados e retira os acentos das letras e palavras

lista_train = tweet_tokenizer.tokenize(train_limpo)    # Separa os emojis das palavras, e cria uma lista
                                                       # em que cada palavra ou emoji é um elemento            
           
tabela_t = pd.Series(lista_train).value_counts()       # Faz uma tabela com as frequências de cada palavra na lista acima

In [5]:
#----------------------------------Selecionando tweets de treinamento relevantes e irrelevantes---------------------------------

linhas_train_relev = train['Classificação'] == 'Relevante'
df_train_relev = train.loc[linhas_train_relev, :]

linhas_train_irrelev = train['Classificação'] == 'Irrelevante'
df_train_irrelev = train.loc[linhas_train_irrelev, :]

#------------------Juntando todos os tweets de treinamento relevantes e irrelevantes cada um numa só string---------------------

tweets_train_relev = ''
for i in df_train_relev['Treinamento']:
    tweets_train_relev += i

tweets_train_irrelev = ''
for i in df_train_irrelev['Treinamento']:
    tweets_train_irrelev += i

    
#----------------------------------Tornando as strings criadas acima em listas de palavras--------------------------------------

lista_train_relev = tweet_tokenizer.tokenize(tira_acentos(cleanup(tweets_train_relev.lower())))

lista_train_irrelev = tweet_tokenizer.tokenize(tira_acentos(cleanup(tweets_train_irrelev.lower())))

In [6]:
#------------------------------Frequências para os tweets de treino relevantes--------------------------------------------------

tabela_tra = pd.Series(lista_train_relev).value_counts()
tabela_trr = pd.Series(lista_train_relev).value_counts(True)

print('Frequência ABSOLUTA das palavras dos tweets RELEVANTES\n')
print(tabela_tra)

print('\n\nFrequência RELATIVA das palavras dos tweets RELEVANTES\n')
print(tabela_trr)   # tabela printada à título de consulta, pois não será usada posteriormente


#------------------------------Frequências para os tweets de treino irrelevantes------------------------------------------------

tabela_tia = pd.Series(lista_train_irrelev).value_counts()
tabela_tir = pd.Series(lista_train_irrelev).value_counts(True)

print('\n\nFrequência ABSOLUTA das palavras dos tweets IRRELEVANTES\n')
print(tabela_tia)

print('\n\nFrequência RELATIVA das palavras dos tweets IRRELEVANTES\n')
print(tabela_tir)   # tabela printada à título de consulta, pois não será usada posteriormente

Frequência ABSOLUTA das palavras dos tweets RELEVANTES

havaianas    127
de            70
e             63
uma           43
a             37
            ... 
amiga          1
louca          1
internet       1
alto           1
tanque         1
Length: 755, dtype: int64


Frequência RELATIVA das palavras dos tweets RELEVANTES

havaianas    0.069857
de           0.038504
e            0.034653
uma          0.023652
a            0.020352
               ...   
amiga        0.000550
louca        0.000550
internet     0.000550
alto         0.000550
tanque       0.000550
Length: 755, dtype: float64


Frequência ABSOLUTA das palavras dos tweets IRRELEVANTES

havaianas         120
de                 93
e                  73
que                43
a                  37
                 ... 
medo                1
olhando             1
pietro              1
httpstcobmdxkz      1
havaianasmoral      1
Length: 1048, dtype: int64


Frequência RELATIVA das palavras dos tweets IRRELEVANTES

havaianas     

In [16]:
#-------------------------Limpando os tweets de teste e transformando-os em listas linha a linha--------------------------------

teste_em_listas = pd.DataFrame()

for i in range(len(test['Teste'])):
    teste_em_listas.loc[i, 'Teste'] = tira_acentos(cleanup(test['Teste'][i].lower()))
    teste_em_listas.loc[i, 'Teste'] = tweet_tokenizer.tokenize(teste_em_listas.loc[i, 'Teste'])

teste_em_listas['Classificação'] = test['Classificação']
teste_em_listas

,Teste,Classificação
0,"[joshnaku, delicia, perfeito, minecraft, chine...",Relevante
1,"[vendese, foto, dos, meus, pes, feios, com, ma...",Relevante
2,"[rt, ribeirod, 10, nao, vejo, a, hora, de, che...",Relevante
3,"[estou, indo, de, havaiana, eu, tem, problema,...",Relevante
4,"[menina, com, shorts, saia, e, havaianas, 🥰, 🥰...",Relevante
...,...,...
195,"[egirao, aeciodepapelao, mas, a, tv, e, uma, u...",Irrelevante
196,"[todo, mundo, q, for, vir, no, meu, aniversari...",Relevante
197,"[eu, prometi, pra, mim, mesmo, que, eu, nao, i...",Relevante
198,"[eu, sempre, tive, toque, com, havaianas, bran...",Irrelevante


In [18]:
def probabilidade(tweet, tabela, lista):     # Função para realizar o cálculo de probabilidades das palavras
    prob = 1
    for palavra in tweet:
        if not palavra in tabela:           # Se a palavra de teste não consta em treinamento, então de acordo com a suavização
            prob = prob / (len(lista) + len(tabela_t)) # de LaPlace, a probabilidade dessa palavra fica com numerador 1
        else:
            prob = prob * (tabela['%s'%(palavra)] + 1) / (len(lista) + len(tabela_t))
    return prob

def classificador(tweet):   # Compara a probabilidade de ser relevante com não relevante e devolve a resposta
    if probabilidade(tweet, tabela_tra, lista_train_relev) > probabilidade(tweet, tabela_tia, lista_train_irrelev):
        return 'Relevante'
    else:
        return 'Irrelevante'

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [22]:
#-------------------------------------Adicionando uma nova coluna ao dataframe Teste--------------------------------------------

teste_em_listas['Performance'] = 'NaN'
for i in range(len(teste_em_listas)):
    teste_em_listas.loc[i, 'Performance'] = classificador(teste_em_listas.loc[i, 'Teste'])
teste_em_listas

,Teste,Classificação,Performance
0,"[joshnaku, delicia, perfeito, minecraft, chine...",Relevante,Relevante
1,"[vendese, foto, dos, meus, pes, feios, com, ma...",Relevante,Relevante
2,"[rt, ribeirod, 10, nao, vejo, a, hora, de, che...",Relevante,Relevante
3,"[estou, indo, de, havaiana, eu, tem, problema,...",Relevante,Relevante
4,"[menina, com, shorts, saia, e, havaianas, 🥰, 🥰...",Relevante,Relevante
...,...,...,...
195,"[egirao, aeciodepapelao, mas, a, tv, e, uma, u...",Irrelevante,Relevante
196,"[todo, mundo, q, for, vir, no, meu, aniversari...",Relevante,Relevante
197,"[eu, prometi, pra, mim, mesmo, que, eu, nao, i...",Relevante,Relevante
198,"[eu, sempre, tive, toque, com, havaianas, bran...",Irrelevante,Relevante


In [23]:
#-------------------------------------Contando o percentual de acertos----------------------------------------------------------

contador = 0
for i in range(len(teste_em_listas)):
    if teste_em_listas.loc[i, 'Classificação'] == teste_em_listas.loc[i, 'Performance']:
        contador += 1
contador = contador / len(teste_em_listas)
    
contador

0.625

In [26]:
#------------------------Dashboard para análise de sentimento e visualização de dados-------------------------------------------

quadro = pd.crosstab(teste_em_listas['Classificação'], teste_em_listas['Performance'], margins = True, normalize = True)
quadro

Performance,Irrelevante,Relevante,All
Classificação,,,
Irrelevante,0.28,0.265,0.545
Relevante,0.11,0.345,0.455
All,0.39,0.610,1.000


Analisando a tabela acima, é possível ver que obtemos os seguintes valores:
* Verdadeiros positivos (mensagens relevantes e que são classificadas como relevantes): 34,5%
* Falsos positivos (mensagens irrelevantes e que são classificadas como relevantes): 26,5%
* Verdadeiros negativos (mensagens irrelevantes e que são classificadas como irrelevantes): 28%
* Falsos negativos (mensagens relevantes e que são classificadas como irrelevantes): 11%

___
### Concluindo

* Total de ACERTOS do classificador: 62,5%
* Total de ERROS do classificador: 37,5%

## Diferentes cenários:

* Classificação de artigos em um jornal, indicando o tema do artigo por meio do uso de jargões e o tipo de reportagem. 
* Classificação de e-mails, onde o classificador poderia indentificar e-mails e separa-lo como spam ou email importantes, dividindo-os e facilitando a vida do usuário.
* Classificação de artigos científicos em materias, indentificando as palavras chave de cada materia. Um exemplo seria um texto com a palavra "integral", onde, com os filtros certos, seria indentificada como artigo de matemática ou física.


## Melhorias:
* Aplicar filtros para descartar palavras que não serão úteis para o classificador, como conectivos e artigos.
* Aplicar uma função que classifique as frases em frases relevantes negativas e positivas, além de separar palavras que contenham sarcasmo indentificando palavras negativas na mesma frase.
* Aplicar mais categorias para melhorar a eficiência do classificador.

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

https://pypi.org/project/Unidecode/